In [28]:
## Imports

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, r2_score

## Loading the data

In [2]:
dataset = pd.read_csv("Data/ALL_counts.csv", index_col=0)

## Preprocessing

In [34]:
## Check for missing values
missing_values = dataset.isnull().sum().sum()
print(missing_values)

0


In [37]:
## Normalization

# Calculate TPM (assuming counts are in columns)
dataset = dataset.div(dataset.sum(axis=1), axis=0) * 1e6

## Concatenate with labels

In [38]:
## Labels 
metadata = pd.read_csv("Data/ALL_metadata.csv", index_col = 0)
metadata = metadata[~metadata.index.duplicated(keep='first')] ## remove some duplicated rows

## Make sure that rows (samples) in the counts matrix (dataset) are in the same order as the metadata
dataset = dataset.loc[metadata.index]
## Add the class (1/0) as final column to the dataset
dataset['mutated'] = metadata.iloc[:,-1]

## Spatial data simulation

In [39]:
def CreateSTSim(dataset, num_circles=6, grid_size=64, circle_radius=[4, 9], seed=None):
    # Set a seed for reproducibility if provided
    if seed is not None:
        np.random.seed(seed)

    # Create node features and labels
    node_features = np.zeros((grid_size, grid_size, len(dataset.columns) - 1))  # Node features
    node_labels = np.zeros((grid_size, grid_size), dtype=float)  # Node labels
    node_names = list()
    radii = np.random.randint(circle_radius[0], circle_radius[1], size=num_circles)  # random radii to define circle (clone) sizes within circle_radius limits
    circle_midpoints = np.random.randint(circle_radius[0], grid_size - circle_radius[0], size=(num_circles, 2))  # Define random circle midpoints within the grid size

    ## Split data into 1 and 0 class to make further processing faster
    data1 = dataset[dataset.iloc[:, -1] == 1]
    data0 = dataset[dataset.iloc[:, -1] == 0]
    # Assign class labels based on circle/clone location
    for i in range(grid_size):
        for j in range(grid_size):
            ## calculate distance between spot and each clone midpoint
            x = np.array([i, j])
            distances = np.linalg.norm(x - circle_midpoints, axis=1)
            if np.any(distances <= radii):
                ## if spot is within any clone (i.e., closer to the midpoint of the clone than the circle radius), assign class 1
                node_labels[i, j] = 1
                random_index = np.random.choice(len(data1.index))
                node_features[i, j] = data1.iloc[random_index, :-1].values
                node_names.append(data1.index[random_index])
            else:
                ## spot is outside a clone: class 0
                random_index = np.random.choice(len(data0.index))
                node_features[i, j] = data0.iloc[random_index, :-1].values
                node_names.append(data0.index[random_index])
    return node_features, node_labels, np.array(node_names).reshape(grid_size,grid_size)


In [40]:
grid_size = 65
features, labels, samples = CreateSTSim(dataset, num_circles = 6, grid_size = grid_size, circle_radius = [4,9],  seed = 1) ## Set seed for reproducibility

In [41]:
## make a grid of x and y coordinates
x, y = np.meshgrid(np.arange(grid_size), np.arange(grid_size), indexing='xy')

## Random forest

In [42]:
# Reshape the features to prepare for modeling
reshaped_features = features.reshape(-1, features.shape[2])  # Reshape to (n_samples, num_features)

# Flatten the labels
flattened_labels = labels.flatten()

# Flatten the coordinates
flattened_x = x.flatten()
flattened_y = y.flatten()

# Concatenate features and coordinates
combined_data = np.column_stack((reshaped_features, flattened_x, flattened_y))

In [43]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(combined_data, flattened_labels, test_size=0.3, random_state=42)

In [44]:
# Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_classifier.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [45]:
# Predict on the test set
predictions = rf_classifier.predict(X_test)

# Evaluate the model

accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")

# Additional evaluation metrics
print("Classification Report:")
print(classification_report(y_test, predictions))

print("Confusion Matrix:")
print(confusion_matrix(y_test, predictions))


Accuracy: 0.8848580441640379
Classification Report:
              precision    recall  f1-score   support

         0.0       0.89      0.99      0.94      1083
         1.0       0.87      0.25      0.39       185

    accuracy                           0.88      1268
   macro avg       0.88      0.62      0.66      1268
weighted avg       0.88      0.88      0.86      1268

Confusion Matrix:
[[1076    7]
 [ 139   46]]
